In [1]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np
import pandas as pd

from regulus.utils import io

In [2]:
import sklearn.linear_model as lm
from sklearn.linear_model import  LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [3]:
layout = go.Layout(
    title='Node data',
    autosize=False,
    width=600,
    height=600,
    margin=dict(
        l=65,
        r=50,
        b=65,
        t=90
    )
)

In [4]:
gauss = io.load('data/gauss4_mc')

In [5]:
node_id = 12
node = gauss.tree.find(lambda _,n: n.id == node_id)
data = node.data.x.copy()
data['z'] = node.data.y

In [6]:
# n12 = pd.read_csv('data/n12.csv')

In [7]:
scaled = data*[10,10,1]

In [8]:
data_pts = go.Scatter3d(
        x = data['x'],
        y = data['y'],
        z = data['z'],
        mode='markers',
        marker=dict(
            size=2,
            color='blue',
            opacity=0.8
        )
    )

In [9]:
scaled_pts = go.Scatter3d(
        x = scaled['x'],
        y = scaled['y'],
        z = scaled['z'],
        mode='markers',
        marker=dict(
            size=2,
            color='blue',
            opacity=0.8
        )
    )

In [10]:
z75 = data.quantile(q=0.75, axis=0)['z']
z75

0.7234462500000001

In [11]:
top = data[data['z'] > z75]

In [12]:
top_pts = go.Scatter3d(
        x = top['x'],
        y = top['y'],
        z = top['z'],
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
                color='red',
                width=0.5
            ),
            opacity=0.8
        )
    )
    

In [13]:
go.FigureWidget(data=[top_pts], layout=layout)

FigureWidget({
    'data': [{'marker': {'line': {'color': 'red', 'width': 0.5}, 'opacity': 0.8, 'size': 2},
  …

In [14]:
go.FigureWidget(data=[data_pts], layout=layout)

FigureWidget({
    'data': [{'marker': {'color': 'blue', 'opacity': 0.8, 'size': 2},
              'mode': 'ma…

In [15]:
# mesh_3d = np.append(mesh, v.reshape(nx*ny,1), 1)

In [16]:
# mesh_pts = go.Scatter3d(
#         x = mesh_3d[:,0],
#         y = mesh_3d[:,1],
#         z = mesh_3d[:,2],
#         mode='markers',
#         marker=dict(
#             size=2
#         )
#     )

In [17]:
nx = 20
ny = 10

In [18]:
def compute(degree, regression, scale='RdBu', pts=data):
    model = make_pipeline(PolynomialFeatures(degree), regression)
    model.fit(pts[['x', 'y']], pts['z'])
    print('score', model.score(pts[['x', 'y']], pts['z']), 
          'mse:', mean_squared_error(model.predict(pts[['x', 'y']]), pts['z']))

    sx = np.linspace( pts['x'].min(), pts['x'].max(), nx)
    sy = np.linspace( pts['y'].min(), pts['y'].max(), ny)
    m = np.array(np.meshgrid(sx, sy))
    mesh = m.T.reshape(-1,2)
    v = model.predict(mesh)
    z = v.reshape(nx, ny).T
    return go.Surface(x=sx, y=sy, z=z, colorscale=scale)

In [19]:
go.FigureWidget(data=
    [
        data_pts, 
        compute(3, LinearRegression()),
#         compute(2, LinearRegression(), pts=top),
#         compute(3, LinearRegression()),
#         compute(4, LinearRegression()),
#         compute(2, Ridge(alpha=1)),
        compute(3, lm.TheilSenRegressor(random_state=42), scale='YlGnBu'),
#         compute(2, lm.RANSACRegressor(random_state=42), scale='greens'),
#         compute(2, lm.HuberRegressor(), scale='YlGnBu'),
#         compute(2, Ridge(alpha=10)),
#         compute(2, lm.Lars(normalize=False), scale='YlGnBu'),
#         compute(2, lm.RidgeCV(), scale='Blues')
    ], layout=layout)

score 0.8679409828033153 mse: 0.007751879366744974
score 0.7882038688301366 mse: 0.012432457048554293


FigureWidget({
    'data': [{'marker': {'color': 'blue', 'opacity': 0.8, 'size': 2},
              'mode': 'ma…

In [20]:
go.FigureWidget(data=
    [
        data_pts, 
#         compute(2, Ridge(alpha=10)),
        compute(2, Ridge(alpha=0)),
    ], layout=layout)

score 0.7538324455227685 mse: 0.014450063515708387


FigureWidget({
    'data': [{'marker': {'color': 'blue', 'opacity': 0.8, 'size': 2},
              'mode': 'ma…

In [21]:
go.FigureWidget(data=
    [
        scaled_pts, 
        compute(2, Ridge(alpha=10), pts=scaled),
        compute(2, Ridge(alpha=0), pts=scaled),
    ], layout=layout)

score 0.728430093490169 mse: 0.01594118447638478
score 0.7538324455227683 mse: 0.014450063515708393


FigureWidget({
    'data': [{'marker': {'color': 'blue', 'opacity': 0.8, 'size': 2},
              'mode': 'ma…